<a href="https://colab.research.google.com/github/JarekMaleszyk/data-science-project-sandbox/blob/main/Apache_Spark_DataFrames_Project_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark DataFrames Project Exercise

In [50]:
try:
  import pyspark
except:
  !pip install pyspark
  import pyspark
finally:
  from pyspark.sql import SparkSession

In [51]:
import os.path
FILE_PATH = "/content/walmart/walmart_stock.csv"

if not os.path.isfile(FILE_PATH):
  !rm -rf /content/jsondata
  !wget -P "/content/walmart/" "https://raw.githubusercontent.com/JarekMaleszyk/data-science-project-sandbox/refs/heads/main/walmart_stock.csv"

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [52]:
sparkSession = SparkSession.builder.appName('Spark DataFrames Project Exercise').getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [53]:
walmart_data_df = sparkSession.read.csv(FILE_PATH,
                                        header=True,
                                        inferSchema=True)

#### What are the column names?

In [54]:
walmart_data_df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [55]:
walmart_data_df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [56]:
#walmart_data_df.head(5)
for row in walmart_data_df.head(5):
  print(row)

Row(Date=datetime.date(2012, 1, 3), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)
Row(Date=datetime.date(2012, 1, 4), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)
Row(Date=datetime.date(2012, 1, 5), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)
Row(Date=datetime.date(2012, 1, 6), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)
Row(Date=datetime.date(2012, 1, 9), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)


#### Use describe() to learn about the DataFrame.

In [57]:
walmart_data_df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [58]:
from pyspark.sql.functions import col
from pyspark.sql.types import (StructType, StructField, StringType, IntegerType,
                               ArrayType, DoubleType, BooleanType, DecimalType)

In [59]:
walmart_data_df_describe = walmart_data_df.describe()

In [60]:
walmart_data_df_describe.select(
    col('summary').cast(StringType()),
    col('Open').cast(DecimalType(18,2)), #castujemy na decimal
    col('High').cast(DecimalType(18,2)),
    col('Low').cast(DecimalType(18,2)),
    col('Close').cast(DecimalType(18,2)),
    col('Volume').cast(DecimalType(18,2)),
    col('Adj Close').cast(DecimalType(18,2))
    ).show()

+-------+-------+-------+-------+-------+-----------+---------+
|summary|   Open|   High|    Low|  Close|     Volume|Adj Close|
+-------+-------+-------+-------+-------+-----------+---------+
|  count|1258.00|1258.00|1258.00|1258.00|    1258.00|  1258.00|
|   mean|  72.36|  72.84|  71.92|  72.39| 8222093.48|    67.24|
| stddev|   6.77|   6.77|   6.74|   6.76| 4519780.84|     6.72|
|    min|  56.39|  57.06|  56.30|  56.42| 2094900.00|    50.36|
|    max|  90.80|  90.97|  89.25|  90.47|80898100.00|    84.91|
+-------+-------+-------+-------+-------+-----------+---------+



In [61]:
from pyspark.sql.functions import format_number
result = walmart_data_df.describe()

result.select(
    result['summary'],
    format_number(result['Open'].cast('float'),2).alias('Open'), #to są dalej stringi
    format_number(result['High'].cast('float'),2).alias('High'),
    format_number(result['Low'].cast('float'),2).alias('Low'),
    format_number(result['Close'].cast('float'),2).alias('Close'),
    format_number(result['Volume'].cast('float'),2).alias('Volume'),
    format_number(result['Adj Close'].cast('float'),2).alias('Adj Close'),
).show()

+-------+--------+--------+--------+--------+-------------+---------+
|summary|    Open|    High|     Low|   Close|       Volume|Adj Close|
+-------+--------+--------+--------+--------+-------------+---------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00| 1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|    67.24|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|     6.72|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|    50.36|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096.00|    84.91|
+-------+--------+--------+--------+--------+-------------+---------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [62]:
walmart_data_df2 = walmart_data_df.withColumn('HV Ratio', walmart_data_df['High']/walmart_data_df['Volume']) #withColumn doda kolumnę
walmart_data_df2.select(col('HV Ratio').cast(DecimalType(18,14))).show()

+----------------+
|        HV Ratio|
+----------------+
|0.00000481971465|
|0.00000629084861|
|0.00000466941299|
|0.00000736733846|
|0.00000891560478|
|0.00000864447744|
|0.00000935182842|
|0.00000829141562|
|0.00000771221210|
|0.00000707176482|
|0.00001015495466|
|0.00000657635415|
|0.00000590145296|
|0.00000854767946|
|0.00000842070951|
|0.00001041448342|
|0.00000831607541|
|0.00000972118381|
|0.00000802943603|
|0.00000630743226|
+----------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [63]:
walmart_data_df.orderBy(walmart_data_df['High'].desc()).head(1)[0][0] # jeden wynik, pierwszy wiersz, pierwsza kolumna

datetime.date(2015, 1, 13)

#### What is the mean of the Close column?

In [64]:
from pyspark.sql.functions import mean
walmart_data_df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [65]:
from pyspark.sql.functions import min, max

In [66]:
walmart_data_df.select(min('Volume').alias('Min_Volume'), max('Volume').alias('Max_Volume')).show()

+----------+----------+
|Min_Volume|Max_Volume|
+----------+----------+
|   2094900|  80898100|
+----------+----------+



#### How many days was the Close lower than 60 dollars?

In [67]:
walmart_data_df.filter('Close < 60').count()

81

In [68]:
walmart_data_df.filter(walmart_data_df['Close'] < 60).count()

81

In [69]:
from pyspark.sql.functions import count
result = walmart_data_df.filter(walmart_data_df['Close'] < 60)
result.select(count('Close').alias('Cnt_Close')).show()

+---------+
|Cnt_Close|
+---------+
|       81|
+---------+



#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [70]:
((walmart_data_df.filter('High > 80').count()*1.0 / walmart_data_df.count()) * 100).__round__(2)

9.14

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [71]:
from pyspark.sql.functions import corr
walmart_data_df.select(corr('High', 'Volume').alias('High_Volume_Corr').cast(DecimalType(18,6))).show()

+----------------+
|High_Volume_Corr|
+----------------+
|       -0.338433|
+----------------+



#### What is the max High per year?

In [73]:
from pyspark.sql.functions import year
walmart_data_df_year = walmart_data_df.withColumn('Year', year(walmart_data_df['Date'])) #dodać kolumnę Year
walmart_data_df_year_agg = walmart_data_df_year.groupBy('Year').max()
walmart_data_df_year_agg.select('Year', 'max(High)').orderBy('Year').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months.

In [75]:
from pyspark.sql.functions import month
walmart_data_df_month = walmart_data_df.withColumn('Month', month(walmart_data_df['Date']))
walmart_data_df_month_avg = walmart_data_df_month.select(['Month', 'Close']).groupBy('Month').mean()
walmart_data_df_month_avg.select('Month', 'avg(Close)').orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



# Great Job!